# Защита персональных данных клиентов

<font size=4>*Описание проекта:*</font>

Страховой компании "Хоть Потоп" необходимо защитить данные путем шифрования. При этом зашифрованные данные должны быть пригодны для модели машинного обучения для предсказания количества страховых выплат клиенту.

<font size=4>*Задачи проекта:*</font>

 - Провести первичный анализ данных
 - Обосновать возможность умножения матрицы признаков на обратимую матрицу при котором не изменятся качество предсказаний модели линейной регрессии. 
 - Проверить алгоритм умножения матрицы признаков на матрицу-ключ и обучить модель на этих признаках. Сравнить показания метрик MSE и R2 с показаниями модели обученной на исходных данных
 - Усложнить алгоритм добавив несколько ключей шифрования
 - Проверить качество модели на зашифрованных данных и сравнить дешифрованные признаки с исходными
 - Сохранить класс шифрования/дешифрования для последующего использования.

## Загрузка и подготовка данных

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import sys

In [ ]:
try:
  df = pd.read_csv('/datasets/insurance.csv')
except:
  df = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')

Переменная MAX будет равна максимальному значению которое может принимать тип int64

In [ ]:
MAX = sys.maxsize

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [ ]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Переименуем колонки в соответствии со стандартами Python

In [ ]:
df.columns = ['gender', 'age', 'salary', 'family', 'payments']
df.head()

,gender,age,salary,family,payments
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Проверка явных дубликатов:

In [ ]:
df.duplicated().sum()

153

Проверка дубликатов в признаках:

In [ ]:
df.iloc[:,0:4].duplicated().sum()

153

Количество дубликатов в признаках совпадает с общим количеством дубликатов. На обучения модели линейной регрессии это не должно повлиять. 

###Базовая модель

Объявим переменные признаков и целевого признака. Признаки в формате DataFrame пригодятся для сравнения дешифрованных данных.

In [ ]:
X_df = df.drop(['payments'], axis=1)
X = X_df.values
y = df.payments.values

Обучим модель линейной регрессии и посмотрим метрики предсказаний:

In [ ]:
lr = LinearRegression()
lr.fit(X, y)
y_hat = lr.predict(X)

mse_base = mean_squared_error(y, y_hat)
r2_base = r2_score(y, y_hat)

print(f'MSE: {mse_base}')
print(f'R2: {r2_base}')


MSE: 0.1233468894171086
R2: 0.42494550286668


## Умножение матриц

### Теоретическое обоснование

При умножении матрицы признаков на обратимую квадратную матрицу происходит следующее(на примере квадратных матриц $A$ и $B$ размером 3х3):


$$  
\begin{bmatrix}
    a_{11} & a_{12} & a_{13}  \\
    a_{21} & a_{22} & a_{23} \\
    a_{31} & a_{32} & a_{33} 
\end{bmatrix}
*
\begin{bmatrix}
    b_{11} & b_{12} & b_{13}  \\
    b_{21} & b_{22} & b_{23} \\
    b_{31} & b_{32} & b_{33} 
\end{bmatrix}
=
\begin{bmatrix}
    c_{11} & c_{12} & c_{13}  \\
    c_{21} & c_{22} & c_{23} \\
    c_{31} & c_{32} & c_{33} 
\end{bmatrix}
$$

Где:

$$
c_{11} = a_{11}*b_{11} + a_{12}*b_{21} + a_{13}*b_{31}\\
c_{12} = a_{11}*b_{12} + a_{12}*b_{22} + a_{13}*b_{32}\\
\dots\\
c_{33} = a_{31}*b_{13} + a_{32}*b_{23} + a_{33}*b_{33}\\
$$

То есть мы видим что с каждой строкой матрицы $A$ происходят линейные преобразования на основе значений столбцов матрицы $B$. При этом эти изменения одинаковы для каждой строки и каждого элемента матрицы $A$. Таким образом это не должно влиять на результаты предсказаний линейной регрессии. 

Декодировть закодированные признаки можно умножив матрицу закодированных признаков на матрицу обратную матрице $B$, это и будет ключ к дешифрованию:


$X$ - матрица признаков\
$E$ - матрица ключ для шифрования\
$E^{-1}$ - матрица ключ для дешифрования\
$I$ - единичная матрица\
$E * E^{-1} = I$ - произведение матрицы на обратную равно единичной матрице

$$
(X*E)* E^{-1} = X*(E* E^{-1}) = X * I = X
$$

Таким образом мы можем производить над признаками любые линейные преобразования не влияя на качество модели линейной регрессии. 

### Проверка шифрования/дешифрования

Проверим качество модели на исходных данных и на зашифрованных при помощи создания матрицы_ключа:

In [ ]:
state = np.random.RandomState(42)
n = X.shape[1]

Матрица-ключ шифрования:

In [ ]:
E = state.randint(MAX, size=(n,n))
E

array([[6909045637428952499, 8314211556539077902, 4279532810384561223,
        1819927849474927636],
       [2878035897379592313, 2877591057541362902, 1071453510346823114,
        6754757701360545140],
       [1865242737500154727, 3838261603483033730,  379716980844854580,
        8668306688712173911],
       [6132484236315524509, 3916965252892395905, 3354078637317002171,
        3383216058915832992]])

Матрица-ключ для дешифрования(матрица обратная E):

In [ ]:
D = np.linalg.inv(E)

Зашифрованные признаки:

In [ ]:
enc_X = X @ E

Дешифрованные признаки:

In [ ]:
dec_X = enc_X @ D

Дешифрованные признаки преобразуем в датафрейм и сравним их с исходными, округлив значения:

In [ ]:
dec_X_df = pd.DataFrame(dec_X, columns=df.columns[:-1])
dec_X_df = np.abs(round(dec_X_df))
dec_X_df

,gender,age,salary,family
0,1.0,41.0,49600.0,1.0
1,0.0,46.0,38000.0,1.0
2,0.0,29.0,21000.0,0.0
3,0.0,21.0,41700.0,2.0
4,1.0,28.0,26100.0,0.0
...,...,...,...,...
4995,0.0,28.0,35700.0,2.0
4996,0.0,34.0,52400.0,1.0
4997,0.0,20.0,33900.0,2.0
4998,1.0,22.0,32700.0,3.0


Исходыне признаки:

In [ ]:
(dec_X_df == X_df).sum()

gender    5000
age       5000
salary    4960
family    5000
dtype: int64

Мы видим что почти по всем значениям после дешифрования у нас признаки совпали. Исключения лишь в столбце Зарплата. Проверим среднюю ошибку в этих столбцах:

In [ ]:
mean_squared_error(X_df.salary, dec_X_df.salary)

1.4558378780933286e-25

Среднеквадратичная ошибка почти равна нулю, то есть это означает что наш метод шифрования работает на практике. 

Проверим модель линейной регрессии на зашифрованных данных:

In [ ]:
lr.fit(enc_X, y)
y_hat = lr.predict(enc_X)

mse_enc = mean_squared_error(y, y_hat)
r2_enc = r2_score(y, y_hat)

print(f'MSE encoded: {mse_enc}\nR2 encoded: {r2_enc}\n')
print(f'MSE difference: {mse_base-mse_enc}\nR2 difference: {r2_base-r2_enc}')



MSE encoded: 0.12334688941721106
R2 encoded: 0.42494550286620236

MSE difference: -1.0245970738509413e-13
R2 difference: 4.776179451937423e-13


Как мы видим модель обученная на зашифрованных данных показывает такой же результат как и на исходных. Разница в метриках стремится к нулю. 

## Алгоритм преобразования

**Алгоритм**

Для более защищенного шифрования используем три ключа: матрицу на которую будем умножать исходные признаки, матрицу которую будем прибавлять к произведению и матрицу которую будем умножить на сумму матриц:

$E_1$ - первая матрица-множитель\
$E_2$ - вторая матрица-множитель\
$W$ - матрица-слагаемое\

Матрицы ключи дешифрования соответственно будут:\
$D_1 = E_1^{-1}$ - матрица обратная матрице $E_1$ \
$D_2 = E_2^{-1}$ - матрица обратная матрице $E_2$ \
$W$ - матрица-слагаемое(его мы будем вычитать при дешифровке

Алгоритм шифрования выглядит следующим образом:

$$
X_{enc} = (X * E_1 + W) * E_2
$$
Алгоритм дешифрования:
$$
X_{dec} = (X_{enc} * D_2 - W) * D_1
$$

**Обоснование**

Подставим в формулу $X_{enc}$ значение $X_{dec}$ имея ввиду что произведение матрицы на обратную от самой себя равно единичной матрице:
$$
X_{enc} = (X * E_1 + W) * E_2 = ((X_{enc} * D_2 - W) * D_1 * E_1 + W) * E_2 = ((X_{enc} * E_2^{-1} - W) * E_1^{-1} * E_1 + W) * E_2 = ((X_{enc} * E_2^{-1} - W) * I + W) * E_2 = ((X_{enc} * E_2^{-1} - W) * I + W) * E_2 = (X_{enc} * E_2^{-1} - W+ W) * E_2 = X_{enc} * E_2^{-1} * E_2 = X_{enc}
$$

Мы видим что равенство выполняется, значит метод шифрования и дешифрования работает. 

## Проверка алгоритма

Для шифрования и дешифрованию матрицы признаков любого размера создадим класс LinearEncoder:

In [ ]:
class LinearEncoder():

# Класс LinearEncoder шифрует матрицу признаков шифрованием Хилла. 
# Метод encode принимает на вход признаки для шифрования, возвращает зашифрованные признаки. 
# Метод decode возвращает дешифрованные признаки. 
  

  def encode(self, X):
    MAX = sys.maxsize
    state1 = np.random.RandomState(42)
    state2 = np.random.RandomState(43)
    self.n = X.shape[1]
    self.m = X.shape[0]
    self.E1 = state1.randint(MAX, size=(self.n, self.n))
    self.E2 = state2.randint(MAX, size=(self.n, self.n))
    self.D1 = np.linalg.inv(self.E1)
    self.D2 = np.linalg.inv(self.E2)
    self.W = state1.randn(self.m, self.n)

    enc_X = (X @ self.E1 + self.W) @ self.E2

    return enc_X
  
  def decode(self, enc_X):
    dec_X = ((enc_X @ self.D2) - self.W) @ self.D1
    return dec_X

  

Создадим объект класса и зашифруем признаки клиентов страховой компании:

In [ ]:
encoder = LinearEncoder()
X_enc = encoder.encode(X)

Обучим модель линейной регрессии и сравним ее метрики с метриками базовой модели, обученной на незашифрованных данных:

In [ ]:
lr = LinearRegression()
lr.fit(X_enc, y)
y_hat = lr.predict(X_enc)

mse_enc = mean_squared_error(y, y_hat)
r2_enc = r2_score(y, y_hat)

print(f'MSE encoded: {mse_enc}\nR2 encoded: {r2_enc}\n')
print(f'MSE difference: {mse_base-mse_enc}\nR2 difference: {r2_base-r2_enc}')



MSE encoded: 0.12334688941895945
R2 encoded: 0.4249455028580512

MSE difference: -1.8508528043525985e-12
R2 difference: 8.628764369689179e-12


Очеувидно, модель предсказывает с такой же точностью как и модель обученная на исходных данных. Ошибка между предсказаниями почти равна нулю. 

Дешифруем матрицу признаков методом decode:

In [ ]:
X_dec = encoder.decode(X_enc)
X_dec_df = pd.DataFrame(X_dec, columns=X_df.columns)

Проверим разницу в датафреймах:

In [ ]:
X_dec_df = np.abs(round(dec_X_df))
(dec_X_df == X_df).sum()

gender    5000
age       5000
salary    4960
family    5000
dtype: int64

Результат получился такой же как и при простом умножении на обратимую матрицу. 

## Итоговый вывод:

Было проведено исследование данных клиентов страховой компании. Для защиты персональных данных было предложено два алгоритма шифрования данных по принципу шифрования Хилла:

 - Первый: с одним ключем шифрования
 - Второй: с тремя ключами шифрования.

Для большей сохранности персональных предлагается использовать второй алгоритм так как большее число ключей снижает вероятность расшифровки со стороны. 

Так же было проведено исследования качества моделей линейной регрессии для предсказания количества страховых случаев для каждого клиента на основании его данных(исходных и закодированных при помощи алгоритма шифрования). 
Модели и в том и в другом случае показали одинаковые результаты. Что говорит о том, что для моделей линейных регрессий шифрование признаков таким способом можно смело применять для сохранения конфиденциальности. 